In [ ]:
# Install TPU support for PyTorch
!pip install torch_xla torch==2.0

# TPU Imports
import torch_xla
import torch_xla.core.xla_model as xm
import torch_xla.distributed.xla_multiprocessing as xmp

from unsloth import FastLanguageModel
import torch

# Set up TPU device
device = xm.xla_device()


In [ ]:
# Load Model and Tokenizer
max_seq_length = 2048 # Choose any!
dtype = None # None for auto detection. BFloat16 for TPUs is better
load_in_4bit = False # TPU does not benefit from 4-bit quantization.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-3B-Instruct", # or your custom model
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

# Move model to TPU
model.to(device)

# LoRA fine-tuning setup for TPU
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Number of LoRA attention heads
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # No dropout
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)


In [ ]:
# Tokenizer and chat template setup
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)

# Formatting function for dataset
def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts }

# Load dataset (JSON format)
from datasets import load_dataset
dataset = load_dataset('json', data_files='/content/processed_data.json', split='train')

# Optional standardization if needed
from unsloth.chat_templates import standardize_sharegpt
dataset = standardize_sharegpt(dataset)
dataset = dataset.map(formatting_prompts_func, batched=True)


In [ ]:
# Show samples from dataset
dataset[5]["conversations"]
dataset[5]["text"]


In [ ]:
# TPU-compatible Training Setup
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model=model,  # LLaMA model for fine-tuning
    tokenizer=tokenizer,  # Tokenizer for processing text data
    train_dataset=dataset,  # Training dataset (Q&A pairs)
    dataset_text_field="text",  # Field name in dataset containing text
    max_seq_length=max_seq_length,  # Max sequence length (number of tokens)
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer),  # Collates batches for seq2seq tasks
    dataset_num_proc=2,  # Number of processes to speed up data loading
    packing=False,  # Don't pack short sequences together
    args=TrainingArguments(
        per_device_train_batch_size=8,  # Larger batch size for TPU
        gradient_accumulation_steps=4,  # Accumulate gradients for larger effective batch size
        warmup_steps=5,  # Warmup steps to gradually increase the learning rate
        num_train_epochs=1,  # Set this for 1 full training run
        learning_rate=2e-4,  # Initial learning rate
        bf16=True,  # Use bfloat16 (optimal for TPU)
        logging_steps=1,  # Log metrics every step
        optim="adamw_torch_xla",  # Optimizer for TPU
        weight_decay=0.01,  # Weight decay to prevent overfitting
        lr_scheduler_type="linear",  # Learning rate schedule: linear decay
        seed=3407,  # Set seed for reproducibility
        output_dir="outputs",  # Directory to save model checkpoints and logs
        tpu_num_cores=8,  # TPU cores for distributed training
    ),
)


In [ ]:
# Train only on responses (can be customized based on your dataset)
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|start_header_id|>user<|end_header_id|>\n\n",
    response_part = "<|start_header_id|>assistant<|end_header_id|>\n\n",
)



In [ ]:
# Train model on TPU
trainer_stats = trainer.train()


In [ ]:
# For inference, move model back to TPU
FastLanguageModel.for_inference(model)

# Inference setup with TPU
prompt = "What is the difference between data science and data engineering?"

messages = [
    {"role": "user", "content": prompt},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True,  # Necessary for generation
    return_tensors = "pt",
).to(device)  # Move input to TPU

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt=True)
_ = model.generate(input_ids=inputs, streamer=text_streamer, max_new_tokens=128,
                   use_cache=True, temperature=1.5, min_p=0.1)


In [ ]:
# Save model and tokenizer
model.save_pretrained("lora_model")  # Save model locally in Colab
tokenizer.save_pretrained("lora_model")

# Zip and download model
!zip -r lora_model.zip lora_model/
from google.colab import files
files.download("lora_model.zip")